# 🛫 Text-Based Travel Booking Agent
## ✨ A Conversational AI Agent for Booking Flights and Hotels using OpenAI

## 📌 Project Overview
This notebook demonstrates a minimal but functional text-based travel booking agent.
It supports conversational booking of flights and hotels using a tool-augmented OpenAI agent.

Ideal for:
- Prototyping travel bots
- Showcasing agentic workflows



### Let’s start by installing the necessary libraries

In [40]:
#basic imports
import json
import uuid
import os
from openai import OpenAI


## 🧠 Setting Up the Brain of Your Travel Agent
In this section, we configure the agent’s environment to ensure smooth operation of the travel booking assistant.

Let’s start by setting up the OpenAI API.


In [19]:
# Setup for environment variables
import os

# Setup for environment variables
os.environ["OPENAI_API_KEY"] = "Enter your OPENAI_API_KEY"
# Verify that the API key is set correctly
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("API Key is set successfully!")
else:
    print("API Key is missing! Please check your setup.")



API Key is set successfully!


### ✈️🏨 *Flight & Hotel Inventory Setup*  
_Defining available flights and hotel options for booking_


In [20]:

# Define the data for flights and hotels
flights = {
    "FL123": {"from": "Delhi", "to": "Mumbai", "price": 5000, "seats": 5},
    "FL456": {"from": "Bangalore", "to": "Delhi", "price": 6000, "seats": 3},
    "FL789": {"from": "Delhi", "to": "Chennai", "price": 5500, "seats": 4},
}

hotels = {
    "H001": {"city": "Mumbai", "name": "Sea View Hotel", "rooms": 10, "price_per_night": 3000},
    "H002": {"city": "Delhi", "name": "Royal Stay", "rooms": 5, "price_per_night": 3500},
    "H003": {"city": "Goa", "name": "Beachside Resort", "rooms": 8, "price_per_night": 4000},
}

### 🛠️📦 *Booking Engine Functions*  
_Logic for searching and booking flights & hotels_


In [21]:
# To connect with your data, replace these functions with your database API calls
def searchFlights(from_city, to_city):
    available_flights = {key: value for key, value in flights.items() if value["from"].lower() == from_city.lower() and value["to"].lower() == to_city.lower()}
    return str(available_flights) if available_flights else "No flights available for this route."

def bookFlight(flight_id, num_seats):
    if flights.get(flight_id) and flights[flight_id]["seats"] >= num_seats:
        flights[flight_id]["seats"] -= num_seats
        return f"Successfully booked {num_seats} seats on flight {flight_id}."
    return "Unable to book flight. Not enough seats available or invalid flight ID."

def searchHotels(city):
    available_hotels = {key: value for key, value in hotels.items() if value["city"].lower() == city.lower()}
    return str(available_hotels) if available_hotels else "No hotels available in this city."

def bookHotel(hotel_id, num_rooms):
    if hotels.get(hotel_id) and hotels[hotel_id]["rooms"] >= num_rooms:
        hotels[hotel_id]["rooms"] -= num_rooms
        return f"Booked {num_rooms} rooms at {hotels[hotel_id]['name']}."
    return "Unable to book hotel. Please check availability."

##  Define Function Calling Schema  
_Setting up callable functions for OpenAI to interact with our booking logic_


In [28]:
# Define the tools for OpenAI API
tools = [
    {"type": "function", "function": {"name": "searchFlights", "description": "Search for available flights based on departure city and destination city.", "parameters": {"type": "object", "properties": {"from_city": {"type": "string"}, "to_city": {"type": "string"}}, "required": ["from_city", "to_city"], "additionalProperties": False}, "strict": True}},
    {"type": "function", "function": {"name": "bookFlight", "description": "Book a flight with a specific ID and number of seats.", "parameters": {"type": "object", "properties": {"flight_id": {"type": "string"}, "num_seats": {"type": "integer"}}, "required": ["flight_id", "num_seats"], "additionalProperties": False}, "strict": True}},
    {"type": "function", "function": {"name": "searchHotels", "description": "Search for available hotels in a specific city.", "parameters": {"type": "object", "properties": {"city": {"type": "string"}}, "required": ["city"], "additionalProperties": False}, "strict": True}},
    {"type": "function", "function": {"name": "bookHotel", "description": "Book a hotel with a specific ID and number of rooms.", "parameters": {"type": "object", "properties": {"hotel_id": {"type": "string"}, "num_rooms": {"type": "integer"}}, "required": ["hotel_id", "num_rooms"], "additionalProperties": False}, "strict": True}}
]


## 🧠📲  Create a Function to Handle OpenAI Calls  
_This function processes OpenAI tool calls and routes them to the correct handler._


In [29]:

# Function to execute the correct tool based on the tool name
def executeToolCall(toolCall):
    tool = toolCall.function
    args = json.loads(tool.arguments)

    if tool.name == "searchFlights":
        return searchFlights(args["from_city"], args["to_city"])
    elif tool.name == "bookFlight":
        return bookFlight(args["flight_id"], args["num_seats"])
    elif tool.name == "searchHotels":
        return searchHotels(args["city"])
    elif tool.name == "bookHotel":
        return bookHotel(args["hotel_id"], args["num_rooms"])
    return "Unknown tool call."



## 💬🤖 Setting Up the OpenAI Chat  
_Finally, we bring everything together into an interactive chat loop!_


In [38]:
# Setup OpenAI API client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Initialize message history
messages = [{"role": "system", "content": "You are a helpful travel booking agent."}]

# Main interaction loop

while True:
    userInput = input("You: ")
    messages.append({"role": "user", "content": userInput})
    response = client.chat.completions.create(model="gpt-4o", messages=messages, tools=tools)
    aiMessage = response.choices[0].message
    messages.append(aiMessage)
    if aiMessage.tool_calls:
        for toolCall in aiMessage.tool_calls:
            toolResponse = executeToolCall(toolCall)
            messages.append({"role": "tool", "content": toolResponse, "tool_call_id": toolCall.id})
        finalResponse = client.chat.completions.create(model="gpt-4o", messages=messages, tools=tools)
        print("AI:", finalResponse.choices[0].message.content)
        messages.append(finalResponse.choices[0].message)
    else:
        print("AI:", aiMessage.content)


API Key is set successfully!
You: hi
AI: Hello! How can I assist you with your travel plans today?
You: i want to go mumbai from delhi is there any flight avilabel
AI: There is a flight available from Delhi to Mumbai. Here are the details:

- **Flight ID:** FL123
- **Price:** ₹5,000
- **Available Seats:** 5

Would you like to book this flight or need any more information?
You: sure book for me
AI: How many seats would you like to book for this flight?
You: 2
AI: Your booking is confirmed! You have successfully booked 2 seats on flight FL123 from Delhi to Mumbai. If there’s anything else you need assistance with, feel free to ask!
You: can you suggest hotels in mumbai
AI: I found a hotel option for you in Mumbai:

- **Hotel Name:** Sea View Hotel
- **Available Rooms:** 10
- **Price per Night:** ₹3,000

Would you like to book a room at this hotel or need more information?
You: can you book this for me
AI: How many rooms would you like to book at the Sea View Hotel?
You: 2
AI: Your bookin

## Preparing data for Llumo evaluation

In [54]:
import json
def convert_to_chunks(messages):
    chunks = []
    current_chunk = []

    for msg in messages:
        if isinstance(msg, dict) and msg.get('role') == 'user':
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = [msg]
        else:
            current_chunk.append(msg)

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def save_chunks(chunks, filename):
    serialized_chunks = []

    for chunk in chunks:
        serialized_chunk = []
        for msg in chunk:
            if isinstance(msg, dict):
                serialized_chunk.append(msg)
            else:
                serialized_chunk.append(msg.model_dump())  # Assuming ChatCompletionMessage supports model_dump()
        serialized_chunks.append(serialized_chunk)

    with open(filename, 'w') as f:
        json.dump(serialized_chunks, f, indent=4)

# Example usage:
chunks = convert_to_chunks(messages)
save_chunks(chunks, 'chunks.json')
with open('/content/chunks.json', 'r') as f:
    loaded_chunks = json.load(f)

### 🧪 Calling Llumo Agentic api for evaluating the agent performance

In [59]:
import requests
url = "Llumo Agentic Eval Api url"



payload = {
    "toolNames": {
        "searchFlights": "Search for available flights based on departure city and destination city.",
        "bookFlight": "Book a flight with a specific flight ID and number of seats.",
        "searchHotels": "Search for available hotels in a specific city.",
        "bookHotel": "Book a hotel with a specific hotel ID and number of rooms."
    },
    "conversations": loaded_chunks
}

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer {Your Llumo ai api}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.status_code)
print(response.json())



[{'Chunk': 2, 'user_query': 'hi', 'tool_count': 0, 'tool_names': [], 'tool_outputs': [], 'tool_errors': [], 'answerprovided': 'yes', 'reasoning': 'no actual query was raised by the user.  The user only greeted the assistant.', 'expectedTools': 'No Tool Expected', 'toolSelectionEval': 'Good', 'actionAdvancement': []}, {'Chunk': 3, 'user_query': 'i want to go mumbai from delhi is there any flight avilabel', 'tool_count': 1, 'tool_names': ['searchFlights'], 'tool_outputs': ["{'FL123': {'from': 'Delhi', 'to': 'Mumbai', 'price': 5000, 'seats': 5}}"], 'tool_errors': [{'tool_name': 'searchFlights', 'status': 'Positive', 'reason': 'The tool successfully returned flight details in a JSON-like format.'}], 'answerprovided': 'yes', 'reasoning': "The assistant successfully identified a flight from Delhi to Mumbai and provided relevant details like flight ID, price, and available seats, fulfilling the user's request.", 'expectedTools': ['searchflights'], 'toolSelectionEval': 'Bad', 'actionAdvanceme

In [ ]:
import pandas as pd

# Convert the response JSON to a DataFrame and save it to an Excel file
# Replace "agentEvalInsights.xlsx" with your desired file name
pd.DataFrame(response.json()).to_excel("agentEvalInsights.xlsx", index=False)
